# US Stock News Summarizer

This notebook fetches and summarizes recent news for US stocks (TSLA, TSM).

In [ ]:
# Install required packages (run once)
# !pip install openai feedparser requests python-dotenv

In [ ]:
import feedparser
import requests
from openai import OpenAI
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

In [ ]:
# Initialize OpenAI client
# API key is loaded from .env file
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file. Please add it to your .env file.")

client = OpenAI(api_key=api_key)
print("OpenAI client initialized successfully!")

## 1. Fetch Stock News

Get latest news for specific stock tickers

In [ ]:
def fetch_stock_news(ticker, num_articles=5):
    """
    Fetch latest news for a specific stock ticker from Google News RSS
    
    Args:
        ticker: Stock ticker symbol (e.g., 'TSLA', 'TSM')
        num_articles: Number of articles to fetch
    
    Returns:
        List of article dictionaries
    """
    # Search for stock news using ticker and company context
    url = f'https://news.google.com/rss/search?q={ticker}+stock&hl=en-US&gl=US&ceid=US:en'
    
    feed = feedparser.parse(url)
    
    articles = []
    for entry in feed.entries[:num_articles]:
        article = {
            'ticker': ticker,
            'title': entry.title,
            'link': entry.link,
            'published': entry.published if 'published' in entry else 'N/A',
            'summary': entry.summary if 'summary' in entry else entry.title
        }
        articles.append(article)
    
    return articles

In [ ]:
def fetch_multiple_stocks_news(tickers, num_articles_per_stock=5):
    """
    Fetch news for multiple stock tickers
    
    Args:
        tickers: List of stock ticker symbols
        num_articles_per_stock: Number of articles to fetch per stock
    
    Returns:
        Dictionary with ticker as key and articles list as value
    """
    all_news = {}
    
    for ticker in tickers:
        print(f"Fetching news for {ticker}...")
        articles = fetch_stock_news(ticker, num_articles_per_stock)
        all_news[ticker] = articles
        print(f"Found {len(articles)} articles for {ticker}\n")
    
    return all_news

In [ ]:
# Fetch news for TSLA and TSM
stock_tickers = ['TSLA', 'TSM']
stock_news = fetch_multiple_stocks_news(stock_tickers, num_articles_per_stock=5)

# Display the news
for ticker, articles in stock_news.items():
    print("="*60)
    print(f"{ticker} - Latest News")
    print("="*60)
    for i, article in enumerate(articles, 1):
        print(f"{i}. {article['title']}")
        print(f"   Published: {article['published']}")
        print(f"   Link: {article['link']}")
        print()
    print()

## 2. AI-Powered Stock News Summary

In [ ]:
def summarize_stock_news(ticker, articles):
    """
    Generate an AI summary for stock news
    
    Args:
        ticker: Stock ticker symbol
        articles: List of article dictionaries
    
    Returns:
        Summary string from OpenAI
    """
    # Combine all article titles and summaries
    news_text = "\n\n".join([
        f"Article {i+1}: {article['title']}\n{article['summary']}"
        for i, article in enumerate(articles)
    ])
    
    # Call OpenAI API with stock-specific prompt
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a financial analyst assistant that summarizes stock news. Provide concise, objective summaries highlighting key developments, market sentiment, and potential impacts."
            },
            {
                "role": "user",
                "content": f"Please provide a brief summary of recent news for {ticker} stock based on these articles:\n\n{news_text}"
            }
        ],
        max_tokens=500,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
# Generate summaries for each stock
summaries = {}

for ticker, articles in stock_news.items():
    print(f"Generating AI summary for {ticker}...\n")
    summary = summarize_stock_news(ticker, articles)
    summaries[ticker] = summary
    
    print("="*60)
    print(f"{ticker} - AI SUMMARY")
    print("="*60)
    print(summary)
    print("\n" + "="*60 + "\n")

## 3. Compare Multiple Stocks

In [ ]:
def compare_stocks_summary(stock_summaries):
    """
    Generate a comparative analysis of multiple stocks
    
    Args:
        stock_summaries: Dictionary with ticker as key and summary as value
    
    Returns:
        Comparative analysis string
    """
    # Combine all summaries
    combined_text = "\n\n".join([
        f"{ticker} Summary:\n{summary}"
        for ticker, summary in stock_summaries.items()
    ])
    
    # Call OpenAI for comparative analysis
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a financial analyst. Compare and contrast the news sentiment and key developments for multiple stocks."
            },
            {
                "role": "user",
                "content": f"Please provide a comparative analysis of these stocks based on their recent news:\n\n{combined_text}"
            }
        ],
        max_tokens=600,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
# Generate comparative analysis
print("Generating comparative analysis...\n")
comparison = compare_stocks_summary(summaries)

print("="*60)
print("COMPARATIVE ANALYSIS: TSLA vs TSM")
print("="*60)
print(comparison)
print("="*60)

## 4. All-in-One Function

In [ ]:
def analyze_stocks(tickers, num_articles_per_stock=5, include_comparison=True):
    """
    Complete stock news analysis pipeline
    
    Args:
        tickers: List of stock ticker symbols
        num_articles_per_stock: Number of articles to fetch per stock
        include_comparison: Whether to generate comparative analysis
    
    Returns:
        Dictionary with news, summaries, and optional comparison
    """
    print("🔍 Fetching stock news...\n")
    stock_news = fetch_multiple_stocks_news(tickers, num_articles_per_stock)
    
    print("\n🤖 Generating AI summaries...\n")
    summaries = {}
    for ticker, articles in stock_news.items():
        summary = summarize_stock_news(ticker, articles)
        summaries[ticker] = summary
        
        print("="*60)
        print(f"{ticker} - SUMMARY")
        print("="*60)
        print(summary)
        print("\n")
    
    comparison = None
    if include_comparison and len(tickers) > 1:
        print("\n📊 Generating comparative analysis...\n")
        comparison = compare_stocks_summary(summaries)
        print("="*60)
        print(f"COMPARATIVE ANALYSIS")
        print("="*60)
        print(comparison)
        print("="*60)
    
    return {
        'news': stock_news,
        'summaries': summaries,
        'comparison': comparison
    }

In [ ]:
# Quick usage example - analyze TSLA and TSM
results = analyze_stocks(['TSLA', 'TSM'], num_articles_per_stock=5)

In [ ]:
# You can also analyze different stocks
# results = analyze_stocks(['AAPL', 'MSFT', 'GOOGL'], num_articles_per_stock=3)